<a href="https://colab.research.google.com/github/bhaveshgupta01/BCancerGogo/blob/main/CancerGogoVGGCheckHyperParameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir = '/content/drive/MyDrive/MIASUnited'


In [4]:
import os

def rename_images(root_dir):
    # Iterate through each subdirectory in the root directory
    for subdir in os.listdir(root_dir):
        subdir_path = os.path.join(root_dir, subdir)

        # Check if it's a directory
        if os.path.isdir(subdir_path):
            # Get a list of image files in the subdirectory
            image_files = [f for f in os.listdir(subdir_path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp'))]

            # Rename each image in the subdirectory
            for i, image_file in enumerate(image_files, start=1):
                old_path = os.path.join(subdir_path, image_file)
                new_name = f"{subdir}_{i:03d}.jpg"  # Adjust the format as needed
                new_path = os.path.join(subdir_path, new_name)

                # Rename the image file
                os.rename(old_path, new_path)
                print(f"Renamed: {old_path} -> {new_path}")

# Replace 'your_root_directory' with the actual root directory containing subdirectories
rename_images('/content/drive/MyDrive/MIASUnited')

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from sklearn.model_selection import StratifiedKFold
from kerastuner.tuners import RandomSearch

# Define paths to your dataset
data_dir = '/content/drive/MyDrive/MIASUnited'

# Image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

# Only rescaling for testing set
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load and augment data
datagen = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
)

# Define the base model (VGG16 without top layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

# Custom model for breast cancer detection
def build_model(hp):
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

# K-Fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# RandomSearch with K-Fold cross-validation using keras-tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Adjust the number of trials
    directory='tuner_results',
    project_name='breast_cancer_detection'
)

# Search for the best hyperparameters
tuner.search(datagen, epochs=10, validation_data=datagen, steps_per_epoch=len(datagen), verbose=1)

# Get the best hyperparameters
best_hps = tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
print("Best Hyperparameters: ", best_hps)


Trial 3 Complete [00h 22m 40s]
val_accuracy: 0.8928571343421936

Best val_accuracy So Far: 0.9464285969734192
Total elapsed time: 01h 02m 37s
Best Hyperparameters:  {'learning_rate': 0.001}
